## 1. Import Library

In [ ]:
import geojson
import pandas as pd
import io
import requests
import difflib

## 2. Load Data

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://ws.lioservices.lrc.gov.on.ca/arcgis1071a/rest/services/LIO_OPEN_DATA/LIO_Open09/MapServer/44/query?where=1%3D1&outFields=*&outSR=4326&f=json') as response:
    phu_geojson = json.load(response)

In [ ]:
# get daily Covid-19 data
url = 'https://data.ontario.ca/dataset/1115d5fe-dd84-4c69-b5ed-05bf0c0a0ff9/resource/d1bfe1ad-6575-4352-8302-09ca81f7ddfc/download/cases_by_status_and_phu.csv'
content = requests.get(url).content
df = pd.read_csv(io.StringIO(content.decode('utf-8')), dtype={"PHU_NUM": str})

In [ ]:
df.head()

In [ ]:
current_date = max(df['FILE_DATE'].unique())
df_current = df.loc[df['FILE_DATE'] == current_date]

In [ ]:
df_current.reset_index(inplace=True, drop=True)

In [ ]:
for g in phu_geojson['features']:
    g['geometry']['coordinates'] = g['geometry'].pop('rings')

In [ ]:
for g in phu_geojson['features']:
    g['properties'] = g.pop('attributes')

In [ ]:
for g in phu_geojson['features']:
    g['properties']['PHU_NUM'] = str(g['properties'].pop('PHU_ID'))

In [ ]:
for g in phu_geojson['features']:
    g['id']= g['properties']['PHU_NUM']

In [ ]:
import plotly.express as px

In [ ]:
fig = px.choropleth_mapbox(df_current, geojson = phu_geojson, color="ACTIVE_CASES",
                    locations="PHU_NUM", featureidkey="properties.PHU_NUM",
                    center={"lat": 44.5517, "lon": -79.7073},
                    mapbox_style="carto-positron", zoom=9
                   )
#fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()